In [ ]:
%%bash
pip install numpy-quaternion

In [1]:
import numpy as np
import quaternion
import matplotlib.pyplot as plt

from scipy import optimize

from camera_calibration_parsers import readCalibration
import image_geometry

from tf import transformations

import time
from copy import deepcopy

import cv2

from IPython.display import clear_output

/usr/local/lib/python2.7/dist-packages/quaternion/numba_wrapper.py:29: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from either numbapro or numba.
This means that the code will run MUCH more slowly.
You probably REALLY want to install numba / numbapro.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)


In [2]:
from client_mk1.fast_image import Image
from client_mk1.color import Color
from client_mk1.feat_2d import Circle2D
from client_mk1.searchers import do_linear_search, do_binary_search

In [3]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

# Useful resources

[OpenCV tutorial](http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html#playing-video-from-file)

![](http://wiki.ros.org/image_pipeline/CameraInfo?action=AttachFile&do=get&target=CameraCoords.png)

## Load camera configuration info

In [4]:
_, calibration = readCalibration("../calibration/stabilized_pixel.yaml")
camera = image_geometry.PinholeCameraModel()
camera.fromCameraInfo(calibration)

# Define types for 3D features

In [5]:
class Sphere3D(object):
    """
    Sphere3D represents a sphere known to exist in 3D map-space.
    """
    def __init__(self, center, radius, colorRange):
        self.center = np.array(center)
        self.radius = radius
        self.color = colorRange

        # TODO: this should be a scalar with more reasonable behavior
        self.confidence = True
        
    def project(self, pos, orientation):
        """
        Returns the coordinates of the given point in the camera image
        """
        relative_center = self.center - pos
        relative_center = np.dot(np.transpose(orientation), relative_center)
        
        center2D = camera.project3dToPixel(relative_center)
        x, y, w = relative_center
        radius2D = camera.fx() * self.radius / w
        
        return Circle2D(center2D, radius2D, self.color)


In [8]:
s = greenSphere
s.project(startPos,np.identity(3))

Circle(center=[1098  759], radius=79)

# Define hardcoded map for testing tracking

In [9]:
green = Color((30, 60, 100), (60, 255, 255))
orange = Color((5, 150, 130), (30, 255, 255))
blue = Color((100, 70, 100), (140, 255, 255))
black = Color((0, 0, 0), (255, 200, 150))

# greenCircle = Circle2D((1920/2, 1080/2), 400, green)
# orangeCircle = Circle2D((1920/2, 1080/2), 400, orange)

greenSmall = Circle2D((1100, 700), 130, green)
orangeSmall = Circle2D((600, 700), 130, orange)

greenSphere = Sphere3D((.4,.3,0), .106, green)
orangeSphere = Sphere3D((-.4,.3,0), .106, orange)
blueSphere = Sphere3D((.4,-.3,0), .106, blue)
blackSphere = Sphere3D((-.4,-.3,0), .106, black)


startPos = np.array([.25, 0, -2.4])
startQuat = np.quaternion(1, 0, 0 ,0)


greenCircle = greenSphere.project(startPos, quaternion.as_rotation_matrix(startQuat))
orangeCircle = orangeSphere.project(startPos, quaternion.as_rotation_matrix(startQuat))
blueCircle = blueSphere.project(startPos, quaternion.as_rotation_matrix(startQuat))
blackCircle = blackSphere.project(startPos, quaternion.as_rotation_matrix(startQuat))

# Find 3D poses

In [10]:
def packStateVector(pos, quat):
    return np.array(tuple(pos)+tuple(quaternion.as_float_array(quat).ravel()))

def unpackStateVector(state):
    position = np.array(state[:3])
    quat = np.quaternion(*state[3:])
    return position, quat

In [11]:
packStateVector([-0.07634509, -0.01887565, -2.86295623],np.quaternion(0.999615505058199, -0.0100498077476995, 0.0505565336078942, -0.00213947738153386))

array([ -7.63450900e-02,  -1.88756500e-02,  -2.86295623e+00,
         9.99615505e-01,  -1.00498077e-02,   5.05565336e-02,
        -2.13947738e-03])

In [28]:
def getDistFunc(spheres, foundCircles):
    assert len(spheres) == len(foundCircles)
    def distFunc(state):
        position, quat = unpackStateVector(state)
        """
        Takes a list of 7 numbers. The first three represent translational 
        position, and the last four form a rotation quaternion.
        Returns a list of error values.
        """
        matrix = quaternion.as_rotation_matrix(quat)
        
        errors = [c.distance(s.project(position, matrix).center)
         for s, c in zip(spheres, foundCircles)]
        
        # Note that scipy leastsq requires sufficient number of outputs,
        # hacked here by feeding extra '0's
        return errors + [quat.norm() - 1] + [0]*4
        
        
    return distFunc

In [29]:
quat = np.quaternion(1,2,3,4)

In [30]:
getDistFunc([greenSphere, orangeSphere], [greenSmall, orangeSmall]
           )(packStateVector(startPos, startQuat))

[59.033888572581766, 115.24755962709145, 0.0, 0, 0, 0, 0]

In [53]:
# TODO: massive speed boosts should come from computing
# the Jacobian more sensibly
def findPose(spheres, circles, pos, quat, verbose=False):
    p,cov,infodict,mesg,ier = optimize.leastsq(
        getDistFunc(spheres, circles),
        packStateVector(pos, quat),
#         xtol=1e-2,
        full_output=1)
    if verbose:
        print '{} evaluations needed, final error {}'.format(infodict['nfev'], infodict['fvec'])
    return unpackStateVector(p)

In [55]:
findPose([greenSphere, orangeSphere], [greenSmall, orangeSmall],
         startPos, startQuat, verbose=True)

8 evaluations needed, final error [  59.03388857  115.24755963    0.            0.            0.            0.
    0.        ]


(array([ 0.25,  0.  , -2.4 ]), quaternion(1, 0, 0, 0))

# Capture and handle video

In [40]:
FILE = '../test_data/smooth_1.mp4'
cv2.namedWindow('raw_frame')

In [69]:
cap = cv2.VideoCapture(FILE)

def handleFrame(frame, features, verbose=False):
    image = Image(frame)

    for f in features:
        f.refine(image, verbose=verbose)
    
    return image


def main():
    features = deepcopy([greenCircle, orangeCircle, blueCircle, blackCircle])
    i=0
    while cap.isOpened():
        verbose = i%50 == 0
        verbose = True
#         if verbose:
#             clear_output()
        i += 1
        
        startTime = time.time()
        ret, frame = cap.read()
        readTime = time.time()

        if frame is None or len(frame) <= 0:
            print "End of video file reached"
            break

        # Do work here
        image = handleFrame(frame, features, verbose=verbose)
        
        finishTime = time.time()

        for f in features:
            f.draw(frame)
            
        image.draw(frame)
        scaled = cv2.resize(frame, dsize=None, fx=0.5, fy=0.5)
        cv2.imshow('raw_frame', scaled)
        
        drawnTime = time.time()

        cv2.waitKey(1)
        
        if verbose:
            processingTime = (finishTime-readTime)
            print "{0:.1f}ms reading and drawing image, {1:.1f}ms ({3}fps) processing, {2} pixels accessed\n".format(
                (readTime-startTime + drawnTime-finishTime)*1000, 
                processingTime*1000, 
                len(image.pixelsAccessed),
                int(1/processingTime))
        
main()

17.8ms reading and drawing image, 2.7ms (364fps) processing, 148 pixels accessed

9.3ms reading and drawing image, 4.7ms (211fps) processing, 148 pixels accessed

8.7ms reading and drawing image, 4.5ms (221fps) processing, 148 pixels accessed

11.1ms reading and drawing image, 5.6ms (180fps) processing, 148 pixels accessed

7.3ms reading and drawing image, 5.4ms (184fps) processing, 148 pixels accessed

6.0ms reading and drawing image, 3.1ms (327fps) processing, 148 pixels accessed

6.2ms reading and drawing image, 3.1ms (319fps) processing, 148 pixels accessed

11.3ms reading and drawing image, 6.0ms (165fps) processing, 148 pixels accessed

6.2ms reading and drawing image, 3.5ms (284fps) processing, 148 pixels accessed

6.7ms reading and drawing image, 3.3ms (303fps) processing, 148 pixels accessed

6.4ms reading and drawing image, 2.8ms (355fps) processing, 148 pixels accessed

8.2ms reading and drawing image, 5.6ms (177fps) processing, 148 pixels accessed

10.5ms reading and drawin

In [42]:
blueCircle

Circle(center=[1345  264], radius=94)

# Profile things

In [ ]:
cap = cv2.VideoCapture(FILE)

def bench():
    features = deepcopy([greenSmall])
    for _ in range(60):
        ret, frame = cap.read()
        handleFrame(frame, features)
    
%lprun -r -f doBinarySearch bench()